In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [4]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
df.describe(include="object")

,keyword,location,text
count,7552,5080,7613
unique,221,3341,7503
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...
freq,45,104,10


In [8]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [12]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [15]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [17]:
model_nm='microsoft/deberta-v3-small'

In [18]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [19]:
tokz.tokenize("hey benim adım Kahraman")

['▁hey', '▁be', 'nim', '▁ad', 'ı', 'm', '▁Kah', 'raman']

In [29]:
tokz("hey benim adım Kahraman")

{'input_ids': [1, 11187, 282, 38847, 4214, 23808, 358, 41053, 65793, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [33]:
def tok_func(x):
    return tokz(x["text"])

In [34]:
tok_ds=ds.map(tok_func,batched=True)
tok_ds

  0%|          | 0/8 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'target', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7613
})

In [45]:
tok_ds["input"][:9]

['TEXT: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'TEXT: Forest fire near La Ronge Sask. Canada',
 "TEXT: All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 'TEXT: 13,000 people receive #wildfires evacuation orders in California ',
 'TEXT: Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 'TEXT: #RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 'TEXT: #flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "TEXT: I'm on top of the hill and I can see a fire in the woods...",
 "TEXT: There's an emergency evacuation happening now in the building across the street"]

In [46]:
tok_ds[0]["input"], tok_ds[0]["input_ids"]

('TEXT: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 [1,
  581,
  65453,
  281,
  262,
  18037,
  265,
  291,
  953,
  117831,
  903,
  4924,
  17018,
  43632,
  381,
  305,
  2])

In [48]:
tokz.tokenize(tok_ds[0]["input"])

['▁TEXT',
 ':',
 '▁Our',
 '▁Deeds',
 '▁are',
 '▁the',
 '▁Reason',
 '▁of',
 '▁this',
 '▁#',
 'earthquake',
 '▁May',
 '▁ALL',
 'AH',
 '▁Forgive',
 '▁us',
 '▁all']

In [50]:
#tokz.vocab["_Our"]

KeyError: '_Our'

In [52]:
eval_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
eval_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [55]:
eval_df.describe(include="object")

,keyword,location,text
count,3237,2158,3263
unique,221,1602,3243
top,deluged,New York,11-Year-Old Boy Charged With Manslaughter of T...
freq,23,38,3


In [59]:
train,validation = tok_ds.train_test_split(0.25, seed=42)
train

'train'

In [58]:
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)
eval_ds

  0%|          | 0/4 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3263
})